In [1]:
import os
# image_path = "../dataset/pizza_steak_sushi/data"
image_path = "../dataset/food101_torch"

train_dir = os.path.join(image_path, "train")
test_dir = os.path.join(image_path, "test")

In [2]:
import torchvision, torch
from torch import nn
# 1. Get pretrained weights for ViT-Base
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT

In [3]:
from data_setup import createDataloader

train_transform = pretrained_vit_weights.transforms()

train_DL, test_DL, _classes = createDataloader(train_dir, test_dir, 32, train_transform, train_transform)

print("total classes:",len(_classes))

total classes: 101


In [4]:


# 2. Setup a ViT model instance with pretrained weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)

# 3. Freeze the base parameters
for parameter in pretrained_vit.parameters():
    parameter.requires_grad = False

# 4. Change the classifier head (set the seeds to ensure same initialization with linear head)
# set_seeds()
pretrained_vit.heads = nn.Linear(in_features=768, out_features=len(_classes)).to(device)
# pretrained_vit # uncomment for model output

In [5]:
from torchinfo import summary

In [6]:
# # Print a summary using torchinfo (uncomment for actual output)
summary(model=pretrained_vit,
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [32, 3, 224, 224]    [32, 101]            768                  Partial
├─Conv2d (conv_proj)                                         [32, 3, 224, 224]    [32, 768, 14, 14]    (590,592)            False
├─Encoder (encoder)                                          [32, 197, 768]       [32, 197, 768]       151,296              False
│    └─Dropout (dropout)                                     [32, 197, 768]       [32, 197, 768]       --                   --
│    └─Sequential (layers)                                   [32, 197, 768]       [32, 197, 768]       --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [32, 197, 768]       [32, 197, 768]       (7,087,872)          False
│    │    └─EncoderBlock (encoder_layer_1)                   [32, 197, 768]       [32, 

In [ ]:
from utils import trainit

loss_fn = nn.CrossEntropyLoss().to(device)
optim = torch.optim.Adam(pretrained_vit.parameters(), lr=1e-3)

trainit(pretrained_vit, train_DL, test_DL, loss_fn, optim, 50, True, device)

  0%|          | 0/50 [00:00<?, ?it/s]